In [36]:
import cv2
import numpy as np

def save_random_frame(video_path, output_path):
    # Read video file
    cap = cv2.VideoCapture(video_path)

    # Choose a random frame number
    frame_number = np.random.randint(0, int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

    # Set the frame number and check if it is read successfully
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()

    if not ret:
        print("Error reading frame. Check if the video file exists and is readable.")
        return

    # Save the frame as an image
    cv2.imwrite(output_path, frame)

    print(f"Random frame saved as {output_path}")

    # Release the video capture object
    cap.release()

# Specify the video file and output image path
video_path = '1.mp4'
output_image_path = 't.jpeg'

# Call the function to save a random frame
save_random_frame(video_path, output_image_path)


Random frame saved as t.jpeg


In [68]:
import cv2
import numpy as np

# Read the input image
image = cv2.imread('t.jpeg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise and improve edge detection
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Use Canny edge detection to find edges in the image
edges = cv2.Canny(blurred, 50, 150)

# Use Hough line transform to detect lines in the image
lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=100)

# Filter lines based on slope and remove duplicates
filtered_lines = []
for line in lines:
    rho, theta = line[0]
    slope = np.abs(np.tan(theta))
    
    # Filter out lines with slopes less than 25 degrees
    if slope < np.tan(np.radians(25)):
        # Check if the line is not a duplicate
        is_duplicate = False
        for existing_line in filtered_lines:
            existing_rho, existing_theta = existing_line[0]
            if np.abs(rho - existing_rho) < 20 and np.abs(theta - existing_theta) < np.radians(5):
                is_duplicate = True
                break
        if not is_duplicate:
            filtered_lines.append(line)
            print(slope)

# Draw the filtered lines on a copy of the original image
result = image.copy()
for line in filtered_lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    cv2.line(result, (x1, y1), (x2, y2), (0, 0, 255), 2)

# Save the result image
cv2.imwrite('res.jpg', result)


0.38386402
0.0
0.40402624


True

True

In [71]:
import cv2
import numpy as np

# Read the input image
image = cv2.imread('k.jfif')

# Define the source points (coordinates of the square)
src_pts = np.array([[100, 100], [200, 100], [200, 200], [100, 200]], dtype=np.float32)

# Define the destination points with different slopes for left and right vertical edges
M1 = 0.4  # Slope for the right vertical edge
M2 = -0.4 # Slope for the left vertical edge
dst_pts = np.array([[100, 100], [200, 100], [200 + M1 * 100, 200], [100 + M2 * 100, 200]], dtype=np.float32)

# Calculate the transformation matrix
transform_matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)

# Apply the transformation to the image
result = cv2.warpPerspective(image, transform_matrix, (image.shape[1], image.shape[0]))

# Save the result image
cv2.imwrite('transformed_image.jpg', result)


True